In [1]:
#研究思路：
#1、统计不同的abnormal_code的次日收益率的分布（找出哪种abnormal_code更容易持续上涨——可加大盘不同区间段研究）
#2、统计次日收益率和net_value、total_value_ratio的关系
#3、统计前3（n）日是否也出现龙虎榜对次日收益率是否有显著影响
#4、加入一些新的特征（前n日成交量、前n日净流入net_value）,把次日收益率做4分类，拿机器学习模型学习

#先获取数据，并计算指标，做成一张大宽表
from jqdata import *
from jqlib.technical_analysis import *
from datetime import datetime, date, timedelta
import pandas as pd
stock_lists = get_all_securities(date='2023-02-13').index.tolist()
stock_lists  = [stock for stock in stock_lists if  (stock.startswith('60') or stock.startswith('00'))]
df=get_billboard_list(stock_lists,end_date='2022-02-13', count=1)
df=df[df['direction']=='ALL']
df=df[['code','day','abnormal_code','abnormal_name','net_value','total_value','amount']]
df.drop_duplicates(subset=['code'],inplace=True)
df['total_value_ratio']=df['total_value']/df['amount']
df

,code,day,abnormal_code,abnormal_name,net_value,total_value,amount,total_value_ratio
0,605398.XSHG,2022-02-11,106004,换手率达20%的证券,-7.463278e+05,3.811531e+07,3.523066e+08,0.108188
20,605305.XSHG,2022-02-11,106002,跌幅偏离值达7%的证券,-2.404597e+07,6.434832e+07,9.658383e+07,0.666243
25,603843.XSHG,2022-02-11,106006,连续三个交易日内收盘价格涨幅偏离值累计达到20%的证券,3.531316e+06,7.363540e+07,1.611376e+08,0.456972
43,603773.XSHG,2022-02-11,106002,跌幅偏离值达7%的证券,-5.478612e+06,4.425666e+07,9.396279e+07,0.471002
47,603721.XSHG,2022-02-11,106003,日价格振幅达到15%的证券,2.289852e+06,2.583740e+07,1.190406e+08,0.217047
65,603477.XSHG,2022-02-11,106006,连续三个交易日内收盘价格涨幅偏离值累计达到20%的证券,-8.997827e+07,5.787628e+08,1.589975e+09,0.364007
75,603322.XSHG,2022-02-11,106009,连续三个交易日内收盘价格跌幅偏离值累计达到15%的证券,-1.122523e+06,2.186269e+06,2.021286e+06,1.081623
79,603229.XSHG,2022-02-11,106002,跌幅偏离值达7%的证券,-6.697399e+07,3.251659e+08,3.879887e+08,0.838081
90,603176.XSHG,2022-02-11,106006,连续三个交易日内收盘价格涨幅偏离值累计达到20%的证券,8.769682e+06,2.413670e+08,2.051224e+09,0.117670
131,603171.XSHG,2022-02-11,106004,换手率达20%的证券,1.174725e+07,1.321724e+08,4.240812e+08,0.311668


In [2]:
#数据处理
def trade_add_days(date,n):
    #前提要保证add之后的日期是在最近一个交易日之前
    trade_days=get_trade_days(start_date='2019-01-01')
    answer=date+ timedelta(days = 1)
    if n==1:
        if answer in trade_days:
            return answer
        else:
            return trade_add_days(answer,1)
    elif answer in trade_days:
        return trade_add_days(answer,n-1)
    else:
        return trade_add_days(answer,n)
#调用方式：
#print(trade_off_days(datetime.date(datetime.strptime("2022-11-18", "%Y-%m-%d")),2))
def test(start_date,end_date):
    target=pd.DataFrame()
    stock_lists = get_all_securities(date=start_date).index.tolist()
    stock_lists  = [stock for stock in stock_lists if  (stock.startswith('60') or stock.startswith('00'))]
    #date=start_date
    while trade_add_days(start_date,3) <= end_date:
        df=get_billboard_list(stock_lists,end_date=start_date, count=1)
        df=df[df['direction']=='ALL']
        df=df[['code','day','abnormal_code','abnormal_name','net_value','total_value','amount']]
        df.drop_duplicates(subset=['code'],inplace=True)
        df['total_value_ratio']=df['total_value']/df['amount']
        df['net_value_ratio']=df['net_value']/df['amount']
        #开盘价
        open=get_bars(list(df['code']),1, unit='1d',fields=['open'],include_now=False, end_dt=trade_add_days(start_date,2), fq_ref_date=None, df=True)
        open.reset_index(inplace=True)
        open.columns=['code','day','open']
        open=open[['code','open']]
        #收盘价
        close=get_bars(list(df['code']),3, unit='1d',fields=['close'],include_now=False, end_dt=trade_add_days(start_date,3), fq_ref_date=None, df=True)
        close.reset_index(inplace=True)
        close.columns=['code','day','close']
        close=close.pivot(index='code',columns='day',values='close')
        close.reset_index(inplace=True)
        close.columns=['code','close_0','close_1','close_2']
        #连接
        df=df.merge(open,on='code',how='inner')
        df=df.merge(close,on='code',how='inner')
        df['net_1']=df['close_1']/df['open']-1
        df['net_2']=df['close_2']/df['open']-1
        df['flag']=[1 if x>=0 else 0 for x in list(df['net_2'])]
        df['count']=[1 for i in list(df['net_2'])]
        #df['net_3']=df['close_3']/df['open']-1
        target=pd.concat([target, df])
        start_date=trade_add_days(start_date,1)
    return target
data=test(datetime.date(datetime.strptime("2022-01-01", "%Y-%m-%d")),datetime.date(datetime.strptime("2023-02-13", "%Y-%m-%d")))

In [3]:
data.head()

,code,day,abnormal_code,abnormal_name,net_value,total_value,amount,total_value_ratio,net_value_ratio,open,close_0,close_1,close_2,net_1,net_2,flag,count
0,605567.XSHG,2021-12-31,106006,连续三个交易日内收盘价格涨幅偏离值累计达到20%的证券,12402610.72,1.724795e+08,997914034.0,0.172840,0.012429,21.50,21.30,23.43,22.80,0.089767,0.060465,1,1
1,605333.XSHG,2021-12-31,106003,日价格振幅达到15%的证券,-16233427.02,5.722253e+07,125357855.0,0.456473,-0.129497,20.46,20.68,20.30,19.43,-0.007820,-0.050342,0,1
2,605289.XSHG,2021-12-31,106004,换手率达20%的证券,7135065.24,9.813033e+07,405403505.0,0.242056,0.017600,41.00,40.10,41.07,42.01,0.001707,0.024634,1,1
3,605033.XSHG,2021-12-31,106004,换手率达20%的证券,38649356.88,1.347986e+08,440627562.0,0.305924,0.087714,26.00,26.71,24.04,21.64,-0.075385,-0.167692,0,1
4,603803.XSHG,2021-12-31,106002,跌幅偏离值达7%的证券,7199159.29,7.201353e+07,507660446.0,0.141854,0.014181,7.44,7.92,7.96,7.53,0.069892,0.012097,1,1


In [4]:
# 连续指标离散化
data['net_value_flag']=[1 if x<=0 else 0 for x in list(data['net_value'])]
data['total_ratio_flag']=[1 if x<=0.31 else 0 for x in list(data['total_value_ratio'])]

In [5]:
# 看一下按照net_value_flag去做离散化，寻找上涨概率的临界值
net_value_distribution=pd.DataFrame(data.groupby(by=['net_value_flag'])['flag','count'].sum())
net_value_distribution['ratio']=net_value_distribution['flag']/net_value_distribution['count']
net_value_distribution

,flag,count,ratio
net_value_flag,,,
0,2438,5350,0.455701
1,2558,5514,0.463910


In [6]:
# 看一下按照total_ratio去做离散化，寻找上涨概率的临界值
total_ratio_distribution=pd.DataFrame(data.groupby(by=['total_ratio_flag'])['flag','count'].sum())
total_ratio_distribution['ratio']=total_ratio_distribution['flag']/total_ratio_distribution['count']
total_ratio_distribution

,flag,count,ratio
total_ratio_flag,,,
0,1644,3356,0.489869
1,3352,7508,0.446457


In [7]:
#看一下不同类型龙虎榜的股票出现次数及次n（3）日的上涨次数及比例
abnormal_code_distribution=pd.DataFrame(data.groupby(by=['abnormal_code'])['flag','count'].sum())
abnormal_code_distribution['code_ratio']=abnormal_code_distribution['flag']/abnormal_code_distribution['count']
abnormal_code_distribution

,flag,count,code_ratio
abnormal_code,,,
106001,1060,2365,0.448203
106002,964,2099,0.459266
106003,597,1405,0.424911
106004,531,1189,0.446594
106005,6,10,0.600000
106006,937,2143,0.437238
106007,296,529,0.559546
106008,46,76,0.605263
106009,45,77,0.584416


In [8]:
# 看一下'net_2','net_value','total_value_ratio' 三者间的皮尔森相关系数（'spearman','pearson','kendall'）
corration=data[['net_2','net_value','total_value_ratio']]
corration.corr(method='pearson')

,net_2,net_value,total_value_ratio
net_2,1.000000,-0.018907,0.009654
net_value,-0.018907,1.000000,-0.029017
total_value_ratio,0.009654,-0.029017,1.000000


In [9]:
data['net_value_ratio']=data['net_value']/data['amount']

In [10]:
#这里只拿本文研究的这两个因子做一下训练合预测（不做深度研究、深度研究文章不便公开）

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
#from sklearn.metrics import plot_roc_curve
from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,precision_recall_fscore_support
feature=['net_value_ratio','total_value_ratio']
X_train,X_test,y_train,y_test=train_test_split(data[feature],data['flag'],test_size=0.3,random_state=0)

In [11]:
#logistic回归方法
#参数设置
model=LogisticRegression()
#模型拟合
model.fit(X_train,y_train)
#模型预测
y_pred_lr=model.predict(X_test)
#评价指标
print ('AUC: %.4f' % metrics.roc_auc_score(y_test,y_pred_lr))
print ('ACC: %.4f' % metrics.accuracy_score(y_test,y_pred_lr))
print ('Recall: %.4f' % metrics.recall_score(y_test,y_pred_lr))
print ('F1-score: %.4f' %metrics.f1_score(y_test,y_pred_lr))
print ('Precesion: %.4f' %metrics.precision_score(y_test,y_pred_lr))
print ('Average_Precesion: %.4f' %metrics.average_precision_score(y_test,y_pred_lr))

print('混淆矩阵为：')
print(metrics.confusion_matrix(y_test,y_pred_lr))

lr=np.array([metrics.roc_auc_score(y_test,y_pred_lr),
            metrics.accuracy_score(y_test,y_pred_lr),
            metrics.recall_score(y_test,y_pred_lr),
            metrics.f1_score(y_test,y_pred_lr),
            metrics.precision_score(y_test,y_pred_lr),
            metrics.average_precision_score(y_test,y_pred_lr)])
print('准确率，召回率以及F1度量如下：')
print(precision_recall_fscore_support(y_test, y_pred_lr,average='binary'))
 

AUC: 0.5045
ACC: 0.5472
Recall: 0.0687
F1-score: 0.1204
Precesion: 0.4856
Average_Precesion: 0.4533
混淆矩阵为：
[[1683 107]
 [1369 101]]
准确率，召回率以及F1度量如下：
(0.4855769230769231, 0.06870748299319727, 0.12038140643623363, None)
